# Take feautres from feature engineering and create 'lookback' feature sets

In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [70]:
#untie local dependency
import os
cwd = os.getcwd().split('\\')
local_repo_path = '\\'.join(cwd[:-1])
raw_data_path = local_repo_path + '\\data\\raw'
processed_data_path = local_repo_path + '\\data\\processed'
feature_path = local_repo_path + '\\data\\features'
submissions_path = local_repo_path + '\\submissions'

In [71]:
output_feature_fname = 'numerics_with_poly'
sj_feature_train = pd.read_csv(feature_path + r'\\sj_train_' + output_feature_fname + '.csv', index_col=0)
sj_feature_test = pd.read_csv(feature_path + r'\\sj_test_' + output_feature_fname + '.csv', index_col=0)
iq_feature_train = pd.read_csv(feature_path + r'\\iq_train_' + output_feature_fname + '.csv', index_col=0)
iq_feature_test = pd.read_csv(feature_path + r'\\iq_test_' + output_feature_fname + '.csv', index_col=0)

In [72]:
sj_feature_train.head()

,0,1,2,3,4,total_cases
0,-1.081814,-1.183554,-0.759571,-1.272727,-0.542448,4
1,-0.559956,-0.583693,-0.214395,-0.272727,-0.541333,5
2,0.006304,-0.005017,-0.214395,0.000000,-0.530953,4
3,-0.061121,-0.053519,0.110260,0.227273,-0.510605,3
4,0.144991,0.146063,0.741194,0.500000,-0.479789,6


In [73]:
iq_feature_train.head()

,0,1,2,3,4,total_cases
0,-0.374283,-0.332172,-1.142356,-0.428571,-0.518389,0
1,-0.271416,-0.242713,-0.637306,-0.357143,-0.515560,0
2,-0.146308,-0.107058,-0.738316,-0.428571,-0.506596,0
3,-1.454387,-1.557470,-1.782087,-4.714286,-0.491497,0
4,-0.961599,-0.962053,-0.940336,-1.571429,-0.470264,0


In [74]:
from math import sqrt
from numpy import concatenate
import matplotlib.pyplot as plt
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Reshape with lookback features - and output data to features folder

# Set lookback

In [75]:
n_lookback = 52

## Train SJ

In [76]:
X_tr_sj_wide = series_to_supervised(sj_feature_train.drop('total_cases', axis = 1), n_lookback, 1)
X_tr_sj_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
X_tr_sj_wide['total_cases'] = sj_feature_train['total_cases']
X_tr_sj_wide.to_csv(feature_path + '\\sj_train_Lookback' + str(n_lookback) + '.csv')
X_tr_sj_wide.shape

(884, 261)

## Test SJ

In [77]:
X_test_sj_wide = sj_feature_train.drop(['total_cases'], axis = 1).iloc[-n_lookback:,:].append(sj_feature_test.drop(['city','weekofyear', 'year'], axis = 1)).reset_index(drop = True)
X_test_sj_wide = series_to_supervised(X_test_sj_wide, n_lookback, 1)
X_test_sj_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
features_out = pd.DataFrame(X_test_sj_wide).reset_index(drop = True)
features_out['city'] = 'sj'
features_out['year'] = sj_feature_test['year']
features_out['weekofyear'] = sj_feature_test['weekofyear']
features_out.to_csv(feature_path + '\\sj_test_Lookback' + str(n_lookback) + '.csv')
X_test_sj_wide = X_test_sj_wide.values.reshape((X_test_sj_wide.shape[0], 1, X_test_sj_wide.shape[1]))
X_test_sj_wide.shape

(260, 1, 260)

## Train IQ

In [78]:
X_tr_iq_wide = series_to_supervised(iq_feature_train.drop('total_cases', axis = 1), n_lookback, 1)
X_tr_iq_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
X_tr_iq_wide['total_cases'] = iq_feature_train['total_cases']
X_tr_iq_wide.to_csv(feature_path + '\\iq_train_Lookback' + str(n_lookback) + '.csv')
X_tr_iq_wide.shape

(468, 261)

## Test IQ

In [79]:
X_test_iq_wide = iq_feature_train.drop('total_cases', axis = 1).iloc[-n_lookback:,:].append(iq_feature_test.drop(['city','weekofyear', 'year'], axis = 1)).reset_index(drop = True)
X_test_iq_wide = series_to_supervised(X_test_iq_wide, n_lookback, 1)
X_test_iq_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
features_out = pd.DataFrame(X_test_iq_wide).reset_index(drop = True)
features_out['city'] = 'iq'
features_out['year'] = iq_feature_test['year']
features_out['weekofyear'] = iq_feature_test['weekofyear']
features_out.to_csv(feature_path + '\\iq_test_Lookback' + str(n_lookback) + '.csv')
X_test_iq_wide = X_test_iq_wide.values.reshape((X_test_iq_wide.shape[0], 1, X_test_iq_wide.shape[1]))
X_test_iq_wide.shape

(156, 1, 260)